In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol, allen_mouse_genes = LoadGeneINFO()

import requests
import SimpleITK as sitk
import pathlib

from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
import alphashape
from alpha_shapes.boundary import Boundary, get_boundaries

/home/jw3514/anaconda3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
MERFISH = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.csv")

/tmp/ipykernel_3216947/4231971372.py:1: DtypeWarning: Columns (2,3,8,9,10,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  MERFISH = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.csv")


In [3]:
MERFISH.head(2)

,cell_label,brain_section_label,cluster_alias,average_correlation_score,feature_matrix_label,donor_label,donor_genotype,donor_sex,x_section,y_section,...,parcellation_category,parcellation_division,parcellation_structure,parcellation_substructure,parcellation_organ_color,parcellation_category_color,parcellation_division_color,parcellation_structure_color,parcellation_substructure_color,ISH_STR
0,1019171907102340387-1,C57BL6J-638850.37,1408,0.596276,C57BL6J-638850,C57BL6J-638850,wt/wt,M,7.226245,4.148963,...,grey,HPF,DG,DG-po,#FFFFFF,#BFDAE3,#7ED04B,#7ED04B,#7ED04B,Dentate gyrus
1,1104095349101460194-1,C57BL6J-638850.26,4218,0.64118,C57BL6J-638850,C57BL6J-638850,wt/wt,M,5.064889,7.309543,...,grey,P,TRN,TRN,#FFFFFF,#BFDAE3,#FF9B88,#FFBA86,#FFBA86,Tegmental reticular nucleus


In [4]:
DG = MERFISH[MERFISH["ISH_STR"]=="Dentate gyrus"]

In [16]:
DG_test = (DG["subclass"].value_counts()/DG["subclass"].shape[0]).to_frame()

In [17]:
DG_test.head(10)

,subclass
037 DG Glut,0.458497
319 Astro-TE NN,0.212523
333 Endo NN,0.057889
327 Oligo NN,0.043993
330 VLMC NN,0.034322
019 L2/3 IT PPP Glut,0.024031
326 OPC NN,0.018605
038 DG-PIR Ex IMN,0.016299
036 HPF CR Glut,0.014244
334 Microglia NN,0.014051


In [20]:
VTA = MERFISH[MERFISH["ISH_STR"]=="Ventral tegmental area"]
VTA_CellComp = (VTA["subclass"].value_counts()/VTA["subclass"].shape[0]).to_frame()

In [21]:
VTA_CellComp.head(10)

,subclass
215 SNc-VTA-RAmb Foxa1 Dopa,0.322677
327 Oligo NN,0.275724
318 Astro-NT NN,0.134865
333 Endo NN,0.085914
195 SNr-VTA Pax5 Npas1 Gaba,0.034965
326 OPC NN,0.028472
331 Peri NN,0.015984
196 PAG-PPN Pax5 Sox21 Gaba,0.013986
201 PAG-RN Nkx2-2 Otx1 Gaba,0.007992
199 PAG-MRN-RN Foxa2 Gaba,0.007992


In [22]:
NAC = MERFISH[MERFISH["ISH_STR"]=="Nucleus accumbens"]
NAC_CellComp = (NAC["subclass"].value_counts()/NAC["subclass"].shape[0]).to_frame()

In [23]:
NAC_CellComp.head(10)

,subclass
061 STR D1 Gaba,0.345278
062 STR D2 Gaba,0.287779
319 Astro-TE NN,0.072781
063 STR D1 Sema5a Gaba,0.060953
333 Endo NN,0.041867
327 Oligo NN,0.034128
060 OT D3 Folh1 Gaba,0.028050
064 STR-PAL Chst9 Gaba,0.026694
081 ACB-BST-FS D1 Gaba,0.011828
334 Microglia NN,0.011565


In [24]:
STR = MERFISH[MERFISH["ISH_STR"]=="Prelimbic area"]
STR_CellComp = (STR["subclass"].value_counts()/STR["subclass"].shape[0]).to_frame()

In [26]:
STR_CellComp.head(20)

,subclass
030 L6 CT CTX Glut,0.160116
319 Astro-TE NN,0.127611
006 L4/5 IT CTX Glut,0.123883
007 L2/3 IT CTX Glut,0.091432
004 L6 IT CTX Glut,0.085732
333 Endo NN,0.074659
022 L5 ET CTX Glut,0.051965
327 Oligo NN,0.033492
053 Sst Gaba,0.031957
052 Pvalb Gaba,0.031135


In [51]:
STR = "Field CA1"
STR = "Mediodorsal nucleus of thalamus"
STR = MERFISH[MERFISH["ISH_STR"]==STR]
STR_CellComp = (STR["subclass"].value_counts()/STR["subclass"].shape[0]).to_frame()

In [52]:
STR_CellComp.head(20)

,subclass
151 TH Prkcd Grin2c Glut,0.350249
318 Astro-NT NN,0.143952
150 CM-IAD-CL-PCN Sema5b Glut,0.129246
327 Oligo NN,0.126968
333 Endo NN,0.087407
154 PF Fzd5 Glut,0.059031
149 PVT-PT Ntrk1 Glut,0.036350
326 OPC NN,0.017709
331 Peri NN,0.016570
334 Microglia NN,0.010874


In [32]:
STR.shape

(56402, 39)

In [35]:
ASD_Bias_MERFISH_Z2_Cell = pd.read_csv("ASD_Bias_MERFISH_Z2_Cell.csv", index_col="STR")

In [39]:
for i, row in ASD_Bias_MERFISH_Z2_Cell.iterrows():
    ASD_Bias_MERFISH_Z2_Cell.loc[i, "STR_name"] = " ".join(i.split("_"))

In [41]:
ASD_Bias_MERFISH_Z2_Cell.head(2)

,EFFECT,REGION,Rank,EFFECT2,STR_name
STR,,,,,
Main_olfactory_bulb,0.629272,Olfactory_areas,1,0.546121,Main olfactory bulb
Olfactory_tubercle,0.514145,Striatum,2,0.474889,Olfactory tubercle


In [43]:
for i, row in ASD_Bias_MERFISH_Z2_Cell.iterrows():
    STR = row["STR_name"]
    STRDF = MERFISH[MERFISH["ISH_STR"]==STR]
    ASD_Bias_MERFISH_Z2_Cell.loc[i, "STR_ncell"] = STRDF.shape[0]

In [46]:
STR2CellNumber = ASD_Bias_MERFISH_Z2_Cell[["REGION", "STR_name", "STR_ncell"]]

In [47]:
STR2CellNumber.head(5)

,REGION,STR_name,STR_ncell
STR,,,
Main_olfactory_bulb,Olfactory_areas,Main olfactory bulb,77675.0
Olfactory_tubercle,Striatum,Olfactory tubercle,30300.0
Anterior_olfactory_nucleus,Olfactory_areas,Anterior olfactory nucleus,37612.0
Subparaventricular_zone,Hypothalamus,Subparaventricular zone,1441.0
Dorsomedial_nucleus_of_the_hypothalamus,Hypothalamus,Dorsomedial nucleus of the hypothalamus,4698.0


In [48]:
STR2CellNumber.to_csv("STR2CellNumber.csv")